In [1]:
import cv2
import numpy as np
import dlib
from imutils import face_utils
from keras.models import load_model
import face_recognition

Using TensorFlow backend.


Taken from https://github.com/vjgpt/Face-and-Emotion-Recognition

In [2]:
def load_image(image_path, grayscale=False, target_size=None):
    pil_image = image.load_img(image_path, grayscale, target_size)
    return image.img_to_array(pil_image)

def load_detection_model(model_path):
    detection_model = cv2.CascadeClassifier(model_path)
    return detection_model

def detect_faces(detection_model, gray_image_array):
    return detection_model.detectMultiScale(gray_image_array, 1.3, 5)

def draw_bounding_box(face_coordinates, image_array, color):
    x, y, w, h = face_coordinates
    cv2.rectangle(image_array, (x, y), (x + w, y + h), color, 2)

def apply_offsets(face_coordinates, offsets):
    x, y, width, height = face_coordinates
    x_off, y_off = offsets
    return (x - x_off, x + width + x_off, y - y_off, y + height + y_off)

def draw_text(coordinates, image_array, text, color, x_offset=0, y_offset=0,
                                                font_scale=0.5, thickness=1):
    x, y = coordinates[:2]
    #cv2.rectangle(image_array, (x + 60, y + y_offset),(x + x_offset, y + y_offset-15), (255, 255, 255), cv2.FILLED)
    cv2.putText(image_array, text, (x + x_offset, y + y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, thickness, cv2.LINE_AA)

def get_colors(num_classes):
    colors = plt.cm.hsv(np.linspace(0, 1, num_classes)).tolist()
    colors = np.asarray(colors) * 255
    return colors

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

def _imread(image_name):
        return cv.imread(image_name)

def _imresize(image_array, size):
        return cv.resize(image_array, size)

def to_categorical(integer_classes, num_classes=2):
    integer_classes = np.asarray(integer_classes, dtype='int')
    num_samples = integer_classes.shape[0]
    categorical = np.zeros((num_samples, num_classes))
    categorical[np.arange(num_samples), integer_classes] = 1
    return categorical

In [3]:
emotion_model_path = './model.hdf5'

In [4]:
labels = {0:'angry',1:'disgust',2:'fear',3:'happy',
                4:'sad',5:'surprise',6:'neutral'}

In [5]:
frame_window = 10
emotion_offsets = (20, 40)

# loading models
detector = dlib.get_frontal_face_detector()
emotion_classifier = load_model(emotion_model_path)

/home/rizwan/anaconda3/envs/acads/lib/python3.6/site-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [6]:
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []

In [7]:
cap = cv2.VideoCapture("./testvdo.mp4")

In [8]:
while cap.isOpened():
    ret, frame = cap.read()
    if frame is None:
            break
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    faces = detector(rgb_image)
    for face_coordinates in faces:
        x1, x2, y1, y2 = apply_offsets(face_utils.rect_to_bb(face_coordinates), emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
        except:
            continue
            
        gray_face = preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)
        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)
        emotion_text = labels[emotion_label_arg]
        
        print(emotion_text)

surprise
surprise
surprise
surprise
surprise
surprise
fear
happy
surprise
happy
surprise
surprise
happy
neutral
neutral
angry
neutral
neutral
angry
angry
fear
happy
sad
fear
sad
angry
angry
angry
angry
surprise
angry
angry
angry
angry
sad
surprise
angry
angry
happy
happy
happy
happy
happy
happy
sad
happy
happy
angry
angry
sad
angry
sad
angry
sad
angry
sad
sad
sad
sad
happy
sad
happy
sad
happy
happy
sad
happy
sad
happy
surprise
happy
surprise
happy
happy
happy
happy
happy
happy
happy
happy
happy
happy
happy
happy
surprise
happy
sad
happy
sad
surprise
happy
sad
surprise
happy
sad
surprise
happy
sad
surprise
happy
sad
sad
happy
happy
happy
happy
happy
happy
angry
happy
happy
sad
happy
sad
happy
happy
sad
happy
sad
sad
happy
happy
sad
happy
sad
sad
happy
happy
sad
sad
happy
sad
happy
sad
happy
sad
happy
happy
sad
happy
sad
sad
happy
happy
sad
happy
sad
happy
sad
happy
sad
sad
happy
happy
happy
happy
happy
sad
happy
sad
happy
sad
happy
sad
happy
sad
happy
happy
sad
sad
happy
sad
happy
sad
h

error: OpenCV(4.2.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
